In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [2]:
num_classes = 2

image_resize = 224

batch_size_training = 5
batch_size_validation = 2

In [3]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

train_generator = data_generator.flow_from_directory(
    'image-classification/oranges/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
    'image-classification/oranges/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 40 images belonging to 2 classes.
Found 12 images belonging to 2 classes.


In [4]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

model = Sequential()
model.add(VGG16(include_top=False, pooling='avg', weights='imagenet'))
model.add(Dense(num_classes, activation='softmax'))
model.layers
model.layers[0].layers
model.layers[0].trainable = False
model.summary()

58892288/58889256 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
8/8 [==============================] - 124s 16s/step - loss: 0.3950 - accuracy: 0.8750 - val_loss: 0.4124 - val_accuracy: 0.9167
Epoch 2/2
8/8 [==============================] - 142s 18s/step - loss: 0.1575 - accuracy: 0.9500 - val_loss: 0.2186 - val_accuracy: 0.9167


In [6]:
model.save('classifier_oranges_vgg16_model.h5')

In [15]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

batch_size_testing = 1

test_generator = data_generator.flow_from_directory(
    'image-classification/oranges/test',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_testing,
    class_mode='categorical')

Found 8 images belonging to 2 classes.


In [16]:
vgg_score = model.evaluate(test_generator,
                      steps=batch_size_testing,
                      callbacks=None,
                      max_queue_size=10,
                      workers=1,
                      use_multiprocessing=False,
                      verbose=1)

1/1 [==============================] - 1s 740ms/step - loss: 0.0494 - accuracy: 1.0000


In [17]:
print("Performance for VGG16 is {:5.2f}% accurate.".format(100*vgg_score[1]))

Performance for VGG16 is 100.00% accurate.


In [30]:
steps_per_epoch_training = len(test_generator)

orange_class = model.predict(test_generator,
                      steps=batch_size_testing,
                      callbacks=None,
                      max_queue_size=10,
                      workers=1,
                      use_multiprocessing=False,
                      verbose=1)

1/1 [==============================] - 1s 969ms/step


In [31]:
print("Performance for VGG16 is {:5.2f}% accurate.".format(100*vgg_score[1]))

Performance for VGG16 is 100.00% accurate.


In [39]:
for i in range(1):
  #print(orange_class[i])
  if orange_class[i][0] < orange_class[i][1]:
    print("* Negative")
  else:
    print("* Positive")

* Positive
